In [3]:
import pickle
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.models import save_model
from keras.models import load_model
import os

X = pickle.load(open("../data/X_data_random.pkl","rb"))
y = pickle.load(open("../data/y_data_random.pkl","rb"))
word2id, id2word = pickle.load(open("../data/word2idandid2word.pkl","rb"))
label2id, id2label = pickle.load(open("../data/label2idandid2label.pkl","rb"))
word2vec_matrix = pickle.load(open("../data/word2vec_matrix.pkl","rb"))


index1 = 50000
index2 = 70000
X_train, X_val, X_test = X[:index1], X[index1:index2], X[index2:]
y_train, y_val, y_test = y[:index1], y[index1:index2], y[index2:]

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

padding_len = 200

with tf.device('/gpu:0'):
    model_input = Input(shape=(padding_len,))
    
    wordembed = Embedding(len(word2vec_matrix), 300, weights=[word2vec_matrix], input_length=padding_len, trainable=False)(model_input)
    convs = []
    filter_size = [2, 3, 4, 5]
    for i in filter_size:
        conv_layer = Conv1D(filters=128, kernel_size=i, activation='relu')(wordembed)
        pool_layer = MaxPooling1D(200-i+1, 1)(conv_layer)
        pool_layer = Flatten()(pool_layer)
        convs.append(pool_layer)
    sen2vec = concatenate(convs, axis=1)
    
    model_output = Dense(len(label2id), activation='softmax')(sen2vec)
    model = Model(inputs=model_input, outputs=model_output)
    model.summary()
    checkpoint = ModelCheckpoint("../PCmodel/TextCNN.h5", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.fit(X_train, y_train, batch_size=200, epochs=10, validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.save("../PCmodel/TextCNN.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     68857800    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 199, 128)     76928       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 198, 128)     115328      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_7 (

In [4]:
y_pred = model.predict(X_test)
y1 = [np.argmax(i) for i in y_test]
y2 = [np.argmax(i) for i in y_pred]

from sklearn import metrics

print('precision:')
print(metrics.precision_score(y1,y2,average='micro'))
print(metrics.precision_score(y1,y2,average='macro'))

print('recall:')
print(metrics.recall_score(y1,y2,average='micro'))
print(metrics.recall_score(y1,y2,average='macro'))

print('f1:')
print(metrics.f1_score(y1,y2,average='micro'))
print(metrics.f1_score(y1,y2,average='macro'))

precision:
0.8007
0.7993109552121705
recall:
0.8007
0.7995786490579342
f1:
0.8007
0.7985853541424279
